In [45]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/world-happiness-report.csv')

In [58]:
df.head(3)
df.tail(3)

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
1946,Zimbabwe,2018,3.616,8.049,0.775,55.6,0.763,-0.068,0.844,0.710,0.212
1947,Zimbabwe,2019,2.694,7.950,0.759,56.2,0.632,-0.064,0.831,0.716,0.235
1948,Zimbabwe,2020,3.160,7.829,0.717,56.8,0.643,-0.009,0.789,0.703,0.346


In [47]:
df.shape

(1949, 11)

In [48]:
df.isnull().sum()

Country name                          0
year                                  0
Life Ladder                           0
Log GDP per capita                   36
Social support                       13
Healthy life expectancy at birth     55
Freedom to make life choices         32
Generosity                           89
Perceptions of corruption           110
Positive affect                      22
Negative affect                      16
dtype: int64

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1949 entries, 0 to 1948
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Country name                      1949 non-null   object 
 1   year                              1949 non-null   int64  
 2   Life Ladder                       1949 non-null   float64
 3   Log GDP per capita                1913 non-null   float64
 4   Social support                    1936 non-null   float64
 5   Healthy life expectancy at birth  1894 non-null   float64
 6   Freedom to make life choices      1917 non-null   float64
 7   Generosity                        1860 non-null   float64
 8   Perceptions of corruption         1839 non-null   float64
 9   Positive affect                   1927 non-null   float64
 10  Negative affect                   1933 non-null   float64
dtypes: float64(9), int64(1), object(1)
memory usage: 167.6+ KB


### 상관계수 확인해보기(...굳이 필요한지 의문)

In [56]:
X1 = df.drop('year', axis=1)
X1.head()

,Country name,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,Afghanistan,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.518,0.258
1,Afghanistan,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.584,0.237
2,Afghanistan,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.618,0.275
3,Afghanistan,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.611,0.267
4,Afghanistan,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.710,0.268


In [59]:
# 상관계수 확인
X1.corr().sum()/(len(X1.columns)) # 관계성이 높지 않음을 확인

Life Ladder                         0.372595
Log GDP per capita                  0.335580
Social support                      0.327501
Healthy life expectancy at birth    0.345006
Freedom to make life choices        0.282400
Generosity                          0.157482
Perceptions of corruption          -0.113073
Positive affect                     0.285601
Negative affect                    -0.051862
dtype: float64

### 데이터 분할 전, 결측치 대체하기

##### 시계열(?) 데이터로 보고 fillna() 이용해서 이전 또는 이후 데이터로 결측치 대체

In [50]:
# 결측 데이터 이전에서 유효한 값으로 채우고, 유효값이 없을 경우에는 이후에 유효한 값으로 채우게 되는 코드
df = df.fillna(method='ffill').fillna(method='bfill')

In [51]:
df.isnull().sum()

Country name                        0
year                                0
Life Ladder                         0
Log GDP per capita                  0
Social support                      0
Healthy life expectancy at birth    0
Freedom to make life choices        0
Generosity                          0
Perceptions of corruption           0
Positive affect                     0
Negative affect                     0
dtype: int64

### 데이터 분할하기

In [70]:
# X, Y 분할
X = df.drop('Country name', axis = 1)
y = df[['Country name']]

In [53]:
X.head()

,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.518,0.258
1,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.584,0.237
2,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.618,0.275
3,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.611,0.267
4,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.710,0.268


In [71]:
y.head()

,Country name
0,Afghanistan
1,Afghanistan
2,Afghanistan
3,Afghanistan
4,Afghanistan


### 클래스 : 레이블 인코딩하기 (...년도도 인코딩 해야하나?)

In [79]:
from sklearn import preprocessing

# 레이블 인코딩 함수 생성
def encode_features(dataDf) :
    # 문자열 카테고리 피처
    features = y[['Country name']]
    for feature in features : # 라벨 인코딩 진행
        le = preprocessing.LabelEncoder()
        le = le.fit(dataDf[feature])
        dataDf[feature] = le.transform(dataDf[feature])
    
    return dataDf

In [94]:
y = encode_features(y)
y.head(20)

<ipython-input-79-bb730fc6e651>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataDf[feature] = le.transform(dataDf[feature])


,Country name
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


### 학습 데이터와 테스트 데이터 분리하기

In [82]:
# 테스트 데이터 세트 : 20%
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [84]:
X_train.head(3)

,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
1031,2013,3.816,7.334,0.673,57.06,0.480,-0.019,0.868,0.734,0.241
299,2012,7.415,10.744,0.948,72.52,0.918,0.290,0.466,0.857,0.229
1227,2008,7.381,10.528,0.944,71.60,0.893,0.298,0.334,0.854,0.232


In [86]:
y_train.head(3)

,Country name
1031,87
299,25
1227,105


### 의사결정 트리 알고리즘 이용해서 학습/예측 수행하기 (학습/테스트 데이터 분리할 때 random_state 설정했는데, 의사결정 분류기에도 동일하게 적용해야 하는지?)

In [87]:
from sklearn.tree import DecisionTreeClassifier

# 객체 생성
dt_clf = DecisionTreeClassifier(random_state=0)

# 학습 수행
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

### 테스트 데이터로 예측 수행

In [88]:
pred = dt_clf.predict(X_test)
pred

array([  1, 128, 122,  16,  28,  11, 124,  37, 125,  37,  49, 130,   6,
       119, 111,  23,  23,  46,  89, 137,  71,   6,  94,   6,   1, 111,
        54,  70, 160, 152, 155, 122, 145, 145,  12,  38,  22, 104,  30,
       100, 157, 103, 105,  13,  27,  80,  83,  75, 110,  64,  51,  73,
       129, 154, 165, 126, 125,  61, 149, 165,   2, 110,  13,   7,  85,
        65, 126, 134,  38, 163, 114, 165,  63,  16,  42,  99,  25,  55,
        24,  26,   4, 146,  34,  92,  44, 165, 148, 162,  24, 164,  74,
        21,   6,  77,   0,   2,   0, 126,   6,  73,  11,  53,  97,  88,
        55, 121,  56, 110, 107,   4,  88,  71,  49, 100, 110,  34,  64,
        43,  56,   9, 156,  83,  62,  46, 114, 119, 161,  34,  21, 153,
        97,  99, 103,  74,   2, 142, 133,  19, 116, 157, 149,  42,  92,
       160, 144,  35, 156,  48,  81, 108, 125, 146, 109,  30, 148, 116,
       111,  46,  19, 100,  38, 134,  55, 106, 162,  27,  33,  96,  92,
       118,  80,  18,  81,  96, 111, 106, 131, 126,  61, 145,  4

In [92]:
# 실제 테스트 데이터 레이블 값과 예측한 값 비교 >> 헐,, 출력 결과물이 비교하기 어려운데,, 이럴 경우 어떻게 하는지,,?
print(pred[:10])
print(y_test[:10])

[  1 128 122  16  28  11 124  37 125  37]
      Country name
18               1
1520           128
1437           122
187             16
333             28
135             11
1465           124
1581           135
1476           125
434             37


### 예측 정확도 평가

In [93]:
from sklearn.metrics import accuracy_score

print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.6051


##### "정확도 : 예측 결과 실제 레이블 값과 얼마나 정확하게 맞는지 평가하는 지표" 인데 굉장히 낮게 나옴.. 이건 무슨 의미인지,,,,ㅠ
##### 데이터가 잘못 됐다는 의미인지, 아니면 다른 모델로 예측 정확도를 평가해야 한다는 의미인지,,,?